# Librerias fundamentales para manipular datos y herramienta de optimizacion

In [10]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

# leer la hoja de excel

In [11]:
data = pd.read_excel("../input/data-para-optimizacion/Datos para Problema de Optimizacin.xlsx")

# Limpiar y separar las tablas que se van a utilizar en el modelo

In [ ]:
demanda_de_cada_ciudad = data.iloc[1:8,1:7]
capacidad_de_produccion_de_cada_fabrica = data.iloc[12:18,1:7]
inventario_inicial = data.iloc[22:25,1:3]
costo_de_produccion = data.iloc[28:34,1:7]
costo_mensual_de_inventario = data.iloc[37:40,1:7]
costo_fijo_de_fabrica_tercerizada = data.iloc[43:47,1:7]
costo_de_transporte  = data.iloc[50:,1:]

demanda_de_cada_ciudad.columns = demanda_de_cada_ciudad.iloc[0]
demanda_de_cada_ciudad = demanda_de_cada_ciudad.iloc[1:]
d_cols = demanda_de_cada_ciudad.columns
demanda_de_cada_ciudad = demanda_de_cada_ciudad.set_index(d_cols[0])

capacidad_de_produccion_de_cada_fabrica.columns = capacidad_de_produccion_de_cada_fabrica.iloc[0]
capacidad_de_produccion_de_cada_fabrica = capacidad_de_produccion_de_cada_fabrica.iloc[1:]
d_cols = capacidad_de_produccion_de_cada_fabrica.columns
capacidad_de_produccion_de_cada_fabrica = capacidad_de_produccion_de_cada_fabrica.set_index(d_cols[0])

inventario_inicial.columns = inventario_inicial.iloc[0]
inventario_inicial = inventario_inicial.iloc[1:]
d_cols = inventario_inicial.columns
inventario_inicial = inventario_inicial.set_index(d_cols[0])

costo_de_produccion.columns = costo_de_produccion.iloc[0]
costo_de_produccion = costo_de_produccion.iloc[1:]
d_cols = costo_de_produccion.columns
costo_de_produccion = costo_de_produccion.set_index(d_cols[0])

costo_mensual_de_inventario.columns = costo_mensual_de_inventario.iloc[0]
costo_mensual_de_inventario = costo_mensual_de_inventario.iloc[1:]
d_cols = costo_mensual_de_inventario.columns
costo_mensual_de_inventario = costo_mensual_de_inventario.set_index(d_cols[0])

costo_fijo_de_fabrica_tercerizada.columns = costo_fijo_de_fabrica_tercerizada.iloc[0]
costo_fijo_de_fabrica_tercerizada = costo_fijo_de_fabrica_tercerizada.iloc[1:]
d_cols = costo_fijo_de_fabrica_tercerizada.columns
costo_fijo_de_fabrica_tercerizada = costo_fijo_de_fabrica_tercerizada.set_index(d_cols[0])

costo_de_transporte.columns = costo_de_transporte.iloc[0]
costo_de_transporte = costo_de_transporte.iloc[1:]
d_cols = costo_de_transporte.columns
costo_de_transporte = costo_de_transporte.set_index(d_cols[0])

# Transformacion de los datos 
Pasar las filas a columnas y viceversa de la tabla costo de transporte, añadirle columna de Lima Terceros, luego cambiar el nombre de sus columnas para añadirles si son propias o de terceros, condersarlas en tres columnas que almacenen toda la info de
 origen, destino y costos de transporte; para finalmente unirla con la tabla de costos de produccion y hallar un costo total


In [ ]:
costo_de_transporte_T = costo_de_transporte.T
costo_de_transporte_T["Lima Terceros"] = costo_de_transporte_T["Lima"]
costo_de_transporte_T = costo_de_transporte_T[sorted(costo_de_transporte_T.columns)]
costo_de_transporte_T.columns = costo_de_produccion.sort_index().index
costo_de_transporte_2 = costo_de_transporte_T.T.reset_index()
costo_de_transporte_2.columns = costo_de_transporte_2.columns.fillna("index")

costo_de_transporte_serializado = pd.melt(costo_de_transporte_2,id_vars=["index"],value_vars=costo_de_transporte_2.columns)
costo_de_transporte_serializado = costo_de_transporte_serializado.rename(columns={"index":"Desde",50:"Hacia","value":"costo de transporte"})

costo_de_transporte_serializado = costo_de_transporte_serializado.merge(costo_de_produccion.iloc[:,0],right_index=True,left_on="Desde",how="left")
costo_de_transporte_serializado = costo_de_transporte_serializado.rename(columns={costo_de_transporte_serializado.columns[-1]:"costo de produccion"})

costo_de_transporte_serializado["costo total de entrega"] = costo_de_transporte_serializado["costo de transporte"]+costo_de_transporte_serializado["costo de produccion"]
costo_de_transporte_serializado = costo_de_transporte_serializado.sort_values(["Desde","Hacia"])

# Definicion del modelo a traves de las librerias de scipy 

- El modelo tiene como funcion objetivo a la variable c que representa el costo de transporte mas el costo de produccion para una de las 30 posibles rutas definidas, A_ub representan los coeficientes de las inecuaciones que poseen las 30 variables posibles, b_ub son los valores a restringir las inecuaciones que representan la maxima capacidad de produccion por ciudad.

- El modelo posee la siguiente definición matemática:   
    <br>${x}_{11}+{x}_{12}+{x}_{13}+{x}_{14}+{x}_{15}+{x}_{16} <= 1080000 $</br>
    <br>${x}_{21}+{x}_{22}+{x}_{23}+{x}_{24}+{x}_{25}+{x}_{26} <= 350000 $</br> 
    <br>${x}_{31}+{x}_{32}+{x}_{33}+{x}_{34}+{x}_{35}+{x}_{36} <= 200000 $</br>
    <br>${x}_{41}+{x}_{42}+{x}_{43}+{x}_{44}+{x}_{45}+{x}_{46} <= 80000 $</br>
    <br>${x}_{51}+{x}_{52}+{x}_{53}+{x}_{54}+{x}_{55}+{x}_{56} <= 10000 $</br>
    <br>${x}_{11}+{x}_{21}+{x}_{31}+{x}_{41}+{x}_{51} = 729005 $</br>
    <br>${x}_{12}+{x}_{22}+{x}_{32}+{x}_{42}+{x}_{52} = 236002 $</br>
    <br>${x}_{13}+{x}_{23}+{x}_{33}+{x}_{43}+{x}_{53} = 244038 $</br>
    <br>${x}_{14}+{x}_{24}+{x}_{34}+{x}_{44}+{x}_{54} = 90655 $</br>
    <br>${x}_{15}+{x}_{25}+{x}_{35}+{x}_{45}+{x}_{55} = 178700 $</br>
    <br>${x}_{16}+{x}_{26}+{x}_{36}+{x}_{46}+{x}_{56} = 129726 $</br>
    <br>${x}_{ij} >= 0$</br>

In [ ]:
def programacion_lineal_para_optimizar_los_costes_de_produccion_y_transporte(mes:int):
    
    A_ub = np.full([5,30],0)
    b_ub = capacidad_de_produccion_de_cada_fabrica.sort_index().iloc[:,mes].values
    A_eq = np.full([6,30],0)

    for i in range(5):
        A_ub[i,6*i:6*i+6] =  1

    for i in range(6):
        A_eq[i,5*i:5*i+5] = 1

    A_eq = A_eq.T.reshape(6,30)
    b_eq = demanda_de_cada_ciudad_descontando_inventario.sort_index().iloc[:,mes].values
    
    c = costo_de_transporte_serializado["costo total de entrega"].values

    resultado = linprog(c=c,method="simplex",
                        A_eq=A_eq,b_eq=b_eq,
                        A_ub=A_ub,b_ub=b_ub,
                        bounds=(0,None))

    return resultado

In [ ]:

almacen_inicial_enero = np.array([4e+4,6e+4,0,0,0,0])

almacen_final_enero = np.array([0e+4,0e+4,0,0,0,0])
almacen_final_febrero = np.array([4e+4,6e+4,0,0,0,0])
almacen_final_marzo = np.array([0e+4,0e+4,0,0,0,0])
almacen_final_abril = np.array([5e+4,7.5e+4,0,0,0,0])
almacen_final_mayo = np.array([0e+4,0e+4,0,0,0,0])

almacen_inicial_febrero = almacen_final_enero
almacen_inicial_marzo = almacen_final_febrero
almacen_inicial_abril = almacen_final_marzo
almacen_inicial_mayo = almacen_final_abril


demanda_de_cada_ciudad_descontando_inventario = demanda_de_cada_ciudad.copy()

demanda_de_cada_ciudad_descontando_inventario.iloc[:,0] -= almacen_inicial_enero
demanda_de_cada_ciudad_descontando_inventario.iloc[:,1] -= almacen_inicial_febrero
demanda_de_cada_ciudad_descontando_inventario.iloc[:,2] -= almacen_inicial_marzo
demanda_de_cada_ciudad_descontando_inventario.iloc[:,3] -= almacen_inicial_abril
demanda_de_cada_ciudad_descontando_inventario.iloc[:,4] -= almacen_inicial_mayo

demanda_de_cada_ciudad_descontando_inventario.iloc[:,0] += almacen_final_enero
demanda_de_cada_ciudad_descontando_inventario.iloc[:,1] += almacen_final_febrero
demanda_de_cada_ciudad_descontando_inventario.iloc[:,2] += almacen_final_marzo
demanda_de_cada_ciudad_descontando_inventario.iloc[:,3] += almacen_final_abril
demanda_de_cada_ciudad_descontando_inventario.iloc[:,4] += almacen_final_mayo

resultado_enero = programacion_lineal_para_optimizar_los_costes_de_produccion_y_transporte(0)
resultado_febrero = programacion_lineal_para_optimizar_los_costes_de_produccion_y_transporte(1)
resultado_marzo = programacion_lineal_para_optimizar_los_costes_de_produccion_y_transporte(2)
resultado_abril = programacion_lineal_para_optimizar_los_costes_de_produccion_y_transporte(3)
resultado_mayo = programacion_lineal_para_optimizar_los_costes_de_produccion_y_transporte(4)

In [ ]:
resultado_mayo

In [ ]:
demanda_de_cada_ciudad_descontando_inventario-demanda_de_cada_ciudad

In [ ]:
demanda_de_cada_ciudad_descontando_inventario

In [ ]:
costo_de_transporte_serializado["produccion enero 2022"] = resultado_enero.x
costo_de_transporte_serializado["produccion febrero 2022"] = resultado_febrero.x
costo_de_transporte_serializado["produccion marzo 2022"] = resultado_marzo.x
costo_de_transporte_serializado["produccion abril 2022"] = resultado_abril.x
costo_de_transporte_serializado["produccion mayo 2022"] = resultado_mayo.x

In [ ]:
tabla_de_resultados = pd.DataFrame(index=["Inventario inicial","Produccion","Envios","Inventario final"])

inventario_inicial_total_enero = almacen_inicial_enero.sum()
produccion_total_enero = costo_de_transporte_serializado.iloc[:,-5].sum()
inventario_final_total_enero = almacen_final_enero.sum()
envios_enero = inventario_inicial_total_enero+produccion_total_enero-inventario_final_total_enero

inventario_inicial_total_febrero = almacen_inicial_febrero.sum()
produccion_total_febrero = costo_de_transporte_serializado.iloc[:,-4].sum()
inventario_final_total_febrero = almacen_final_febrero.sum()
envios_febrero = inventario_inicial_total_febrero+produccion_total_febrero-inventario_final_total_febrero

inventario_inicial_total_marzo = almacen_inicial_marzo.sum()
produccion_total_marzo = costo_de_transporte_serializado.iloc[:,-3].sum()
inventario_final_total_marzo = almacen_final_marzo.sum()
envios_marzo = inventario_inicial_total_marzo+produccion_total_marzo-inventario_final_total_marzo

inventario_inicial_total_abril = almacen_inicial_abril.sum()
produccion_total_abril = costo_de_transporte_serializado.iloc[:,-2].sum()
inventario_final_total_abril = almacen_final_abril.sum()
envios_abril = inventario_inicial_total_abril+produccion_total_abril-inventario_final_total_abril

inventario_inicial_total_mayo = almacen_inicial_mayo.sum()
produccion_total_mayo = costo_de_transporte_serializado.iloc[:,-1].sum()
inventario_final_total_mayo = almacen_final_mayo.sum()
envios_mayo = inventario_inicial_total_mayo+produccion_total_mayo-inventario_final_total_mayo

tabla_de_resultados["Enero"] = [inventario_inicial_total_enero,produccion_total_enero,envios_enero,inventario_final_total_enero]
tabla_de_resultados["Febrero"] = [inventario_inicial_total_febrero,produccion_total_febrero,envios_febrero,inventario_final_total_febrero]
tabla_de_resultados["Marzo"] = [inventario_inicial_total_marzo,produccion_total_marzo,envios_marzo,inventario_final_total_marzo]
tabla_de_resultados["Abril"] = [inventario_inicial_total_abril,produccion_total_abril,envios_abril,inventario_final_total_abril]
tabla_de_resultados["Mayo"] = [inventario_inicial_total_mayo,produccion_total_mayo,envios_mayo,inventario_final_total_mayo]

tabla_de_resultados